In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV

In [2]:
X_full = pd.read_csv('diabetes.csv')

In [3]:
X_full.shape

(768, 9)

768 samples with 9 features

In [4]:
#X_full.info()

there is no catigorical features or missing values

In [5]:
#X_full.describe()

['Pregnancies',Glucose',BloodPressure','SkinThickness','Insulin','BMI'] canot have 0 as value but the min is 0 so replace it with the mdian value

In [6]:
# print('Number of rows missing Glucose: {0}'.format(len(X_full.loc[X_full['Glucose'] == 0])))
# print('Number of rows missing Blood Pressure: {0}'.format(len(X_full.loc[X_full['BloodPressure'] == 0])))
# print('Number of rows missing Insulin: {0}'.format(len(X_full.loc[X_full['Insulin'] == 0])))
# print('Number of rows missing BMI: {0}'.format(len(X_full.loc[X_full['BMI'] == 0])))
# print('Number of rows missing Skin Thickness: {0}'.format(len(X_full.loc[X_full['SkinThickness'] == 0])))
# print('Number of rows missing Age: {0}'.format(len(X_full.loc[X_full['Age'] == 0])))

In [7]:
x = X_full['Glucose'].median()
X_full['Glucose'].replace(0,x,inplace=True)
x = X_full['BloodPressure'].median()
X_full['BloodPressure'].replace(0,x,inplace=True)
x = X_full['Insulin'].median()
X_full['Insulin'].replace(0,x,inplace=True)
x = X_full['BMI'].median()
X_full['BMI'].replace(0,x,inplace=True)
x = X_full['SkinThickness'].median()
X_full['SkinThickness'].replace(0,x,inplace=True)

In [8]:
# figure = plt.figure(figsize =(10,10))
# sns.heatmap(X_full.corr(),annot=True,cmap="YlGnBu")

outcome is highly correlated with ['Glucose','BMI','Age','Pregriancies']

In [9]:
X_full['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

will have to make statified sampling to over come the difference between count of 0 and 1 samples

In [10]:
X_full.isna().sum(axis=0)

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

there is not a number in the data

In [11]:
# len(X_full['Age'].unique())

In [12]:
# figure = plt.figure(figsize =(6,6))
# ax = sns.histplot(x="Age",hue ="Outcome" ,data=X_full,multiple="dodge", shrink=.8)

In [13]:
# plt.hist(X_full['BMI'],bins=100)
# plt.show()

## If woman BMI is:

1. below 18.5 – she is in the underweight range -> 0
1. between 18.5 and 24.9 – she is in the healthy weight range -> 1
1. between 25 and 29.9 – she is in the overweight range -> 2
1. between 30 and 39.9 – she is in the obese range -> 3
1. other -> 4

In [14]:
#![image.png](attachment:c731f680-8300-43bf-8f20-c1b68f09ee63.png)

In [15]:
X_full['BMI_levels']=pd.cut(X_full['BMI'],bins=[0, 18.5, 24.99, 29.99,34.99,39.99,100],labels=[0,1,2,3,4,5])

In [16]:
# plt.hist(X_full['BMI_levels'])
# plt.show()

In [17]:
# figure = plt.figure(figsize =(6,6))
# ax = sns.histplot(x = 'BMI_levels',hue ="Outcome",data=X_full ,multiple="dodge", shrink=.8 )

In [18]:
# plt.hist(X_full['Glucose'],bins=100)
# plt.show()

## For a 2 hour GTT"glucose tolerance test" with 75 g intake
1. a glucose level below 7.8 mmol/L (140 mg/dL) is normal
1. Blood plasma glucose between 7.8 mmol/L (140 mg/dL) and 11.1 mmol/L (200 mg/dL) indicate "impaired glucose tolerance"
1. and levels at or above 11.1 mmol/L at 2 hours confirm a diagnosis of diabetes

In [19]:
X_full['Glucose_levels']=pd.cut(X_full['Glucose'],bins=[0, 140 , 200],labels=[0,1])

In [20]:
# figure = plt.figure(figsize =(6,6))
# ax = sns.histplot(x = 'Glucose_levels',hue ="Outcome",data=X_full,multiple="dodge", shrink=.8 )

In [21]:
X_full.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BMI_levels,Glucose_levels
0,6,148,72,35,30.5,33.6,0.627,50,1,3,1
1,1,85,66,29,30.5,26.6,0.351,31,0,2,0
2,8,183,64,23,30.5,23.3,0.672,32,1,1,1
3,1,89,66,23,94.0,28.1,0.167,21,0,2,0
4,0,137,40,35,168.0,43.1,2.288,33,1,5,0


In [22]:
X_org = X_full.drop(columns=['BMI_levels','Glucose_levels'])

### Splitting the dataset for training & testing and standardizing the data¶


In [23]:
Y = X_full['Outcome']
X = X_full.drop(columns=['Outcome'])
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify = Y)

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## cross validation over multibli models

### SVM

In [25]:
from sklearn import svm
support_vector_classifier = svm.SVC(kernel="linear",).fit(X_train_scaled,y_train)
y_pred = support_vector_classifier.predict(X_test_scaled)
cm = confusion_matrix(y_test,y_pred)
print("Our Accuracy is: ", (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]))
print(accuracy_score(y_test,y_pred))

Our Accuracy is:  0.7142857142857143
0.7142857142857143


In [26]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.75      0.85      0.79       100
           1       0.62      0.46      0.53        54

    accuracy                           0.71       154
   macro avg       0.69      0.66      0.66       154
weighted avg       0.70      0.71      0.70       154



In [27]:
svm_params ={"C":np.arange(1,20)}
svms = svm.SVC(kernel="linear")
svm_cv = GridSearchCV(svms,svm_params,cv=8)
svm_cv.fit(X_train_scaled,y_train)
print(svm_cv.best_score_)
print(svm_cv.best_params_)

0.7671949760765551
{'C': 2}


### LogisticRegression

In [28]:
from sklearn.linear_model import LogisticRegression
LRmodel = LogisticRegression(random_state=0).fit(X_train_scaled,y_train)
LRy_pred = LRmodel.predict(X_test_scaled)
LRcm = confusion_matrix(y_test,LRy_pred)
print("Our Accuracy is: ", (LRcm[0][0]+LRcm[1][1])/(LRcm[0][0]+LRcm[1][1]+LRcm[0][1]+LRcm[1][0]))
print(accuracy_score(y_test,LRy_pred))

Our Accuracy is:  0.7142857142857143
0.7142857142857143


In [29]:
print(classification_report(y_test,LRy_pred))

              precision    recall  f1-score   support

           0       0.76      0.82      0.79       100
           1       0.61      0.52      0.56        54

    accuracy                           0.71       154
   macro avg       0.68      0.67      0.67       154
weighted avg       0.71      0.71      0.71       154



In [30]:
LR_params ={"C":np.arange(1,20)}
LRs = LogisticRegression(random_state=0)
LR_cv = GridSearchCV(LRs,LR_params,cv=8)
LR_cv.fit(X_train_scaled,y_train)
print(LR_cv.best_score_)
print(LR_cv.best_params_)

0.7916097060833903
{'C': 3}


# create dasboard to explain my work

In [31]:
import dash
from dash import html   #for dash layout
from dash import dcc    #data control components

In [32]:
from dash.dependencies import Input,Output,State

In [33]:
import plotly.express as px

In [34]:
my_css_link='https://codepen.io/chriddyp/pen/bWLwgP.css'

In [35]:
app = dash.Dash( external_stylesheets=[my_css_link])

In [36]:
#BANs
Y_vals = X_full['Outcome'].value_counts()
with_out_diap = Y_vals[0]
with_dip = Y_vals[1]
model_acu1 = np.round(LR_cv.best_score_,4)
model_acu2 = np.round(svm_cv.best_score_,4)

In [37]:
# columns names
Variables = X.columns.to_list()
print(Variables)

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'BMI_levels', 'Glucose_levels']


In [38]:
#graph
g= px.histogram(X_full,x= 'Pregnancies',color='Outcome',color_discrete_sequence=["red", "blue"])
#g.show()

In [39]:
#feature importance for SVM

SVM_coef,=svm_cv.best_estimator_.coef_

SVM_fe_impor = {'features':Variables,'values':SVM_coef}
SVM_fe_impor_df = pd.DataFrame(SVM_fe_impor)

print(SVM_fe_impor_df)
SVM_impor = px.bar(SVM_fe_impor_df,x='features',y='values',color_discrete_sequence=['gray'])

                   features    values
0               Pregnancies  0.186666
1                   Glucose  0.491374
2             BloodPressure -0.041400
3             SkinThickness -0.011837
4                   Insulin -0.065337
5                       BMI  0.504495
6  DiabetesPedigreeFunction  0.122654
7                       Age  0.027750
8                BMI_levels -0.187668
9            Glucose_levels  0.458741


In [40]:
#feature importance for LR

LR_coef,=LR_cv.best_estimator_.coef_

LR_fe_impor = {'features':Variables,'values':LR_coef}
LR_fe_impor_df = pd.DataFrame(LR_fe_impor)

print(LR_fe_impor_df)
LR_impor = px.bar(LR_fe_impor_df,x='features',y='values',color_discrete_sequence=['gray'])

                   features    values
0               Pregnancies  0.383182
1                   Glucose  1.377049
2             BloodPressure -0.063161
3             SkinThickness  0.027609
4                   Insulin -0.135163
5                       BMI  0.953303
6  DiabetesPedigreeFunction  0.247949
7                       Age  0.139186
8                BMI_levels -0.245938
9            Glucose_levels -0.156500


In [41]:
app.layout =html.Div([ 
    #Introduction to the dataset and our work
    html.H3('Comparison Between LogisticRegression and SVM on Diabetes Prediction(Only for females above 21 years of Age)', style={'textAlign': 'center'}),
    html.P('''The dataset we will be using is PIMA Indian Diabetes Dataset which contains 8 prediction variables 
    and 1 target variable.''',style={'textAlign': 'center'}),
    html.P(''' The predictor variable is named Outcome which is encoded as 0 and 1 
    where 0 represents Non-Diabetic and 1 represents Diabetic.''',style={'textAlign': 'center'}),
    #Big numbers section
    html.Div([
        html.Div([
            html.H6('Samples with out Diabetes',style={'textAlign': 'center'}),
            html.H4(with_out_diap,style={'textAlign': 'center','color':'blue'})
        ],className="three columns"),
        html.Div([
          html.H6('Samples with Diabetes',style={'textAlign': 'center'}),
          html.H4(with_dip,style={'textAlign': 'center','color':'red'})  
        ],className="three columns"),
        html.Div([
           html.H6('Logistic Regression model accuracy',style={'textAlign': 'center'}),
           html.H4(model_acu1,style={'textAlign': 'center','color':'gray'}) 
        ],className="three columns"),
        html.Div([
           html.H6('SVM model accuracy',style={'textAlign': 'center'}),
           html.H4(model_acu2,style={'textAlign': 'center','color':'gray'}) 
        ],className="three columns")
    ]),
    #grahps Section
    html.Div([
        html.Div([
            html.H6('Features Importance',style={'textAlign': 'center'}),
            dcc.Graph(id='graph1'),
            dcc.Dropdown(id='features_importance',
                         options=[{'label':'SVM','value':'SVM'} ,{'label':'LogisticRegression','value':'LogisticRegression'}],
                         value='SVM'
                        )
        ],className="four columns"),
        html.Div([
            html.H6('Disturbution of each varible with respect to target variable',style={'textAlign': 'center'}),
            dcc.Graph(id='graph2',figure=g),
            dcc.Dropdown(id='features_distru',
                         options=[{'label':i,'value':i} for i in Variables],
                         value=Variables[1]
                        )
 
        ],className="four columns"),
        html.Div([
            html.H6('Predict new sample using our LogisticRegression model',style={'textAlign': 'center'}),
            
            dcc.Input(id="Pregnancies", type="number", placeholder="Pregnancies"),
            dcc.Input(id="Glucose", type="number", placeholder="Glucose"),
            dcc.Input(id="BloodPressure", type="number", placeholder="BloodPressure"),
            dcc.Input(id="SkinThickness", type="number", placeholder="SkinThickness"),
            dcc.Input(id="Insulin", type="number", placeholder="Insulin"),
            dcc.Input(id="BMI", type="number", placeholder="BMI"),
            dcc.Input(id="DiabetesPedigreeFunction", type="number", placeholder="DiabetesPedigreeFunction"),
            dcc.Input(id="Age", type="number", placeholder="Age"),
            dcc.Input(id="BMI_levels", type="number", placeholder="BMI_levels"),
            dcc.Input(id="Glucose_levels", type="number", placeholder="Glucose_levels"),
        
            dcc.Dropdown(id='predict',
                         options=[{'label':'SVM','value':'SVM'} ,{'label':'LogisticRegression','value':'LogisticRegression'}],
                         value='SVM',className='three columns'
                        ),
            html.Div(id='result',className='three columns')
        ],className="four columns")
    ])
#     ,
#     html.Div([
#         html.P('comment with link',style={'textAlign': 'left'})
#     ])
    
    
],className='row')

In [42]:
@app.callback(
        Output(component_id='graph2',component_property='figure'),
        Input('features_distru', 'value')
)
def update_features_distru(feature):
    return px.histogram(X_full,x= feature,color='Outcome',color_discrete_sequence=["red", "blue"])

In [43]:
@app.callback(
        Output(component_id='graph1',component_property='figure'),
        Input('features_importance', 'value')
)
def update_features_importance(Model_name):
    if(Model_name=='SVM'):
        return SVM_impor
    else:
        return LR_impor

In [44]:
@app.callback(
        Output(component_id='result',component_property='children'),
        [Input('predict', 'value'),
         Input('Pregnancies', 'value'),
         Input('Glucose', 'value'),
         Input('BloodPressure', 'value'),
         Input('SkinThickness', 'value'),
         Input('Insulin', 'value'),
         Input('BMI', 'value'),
         Input('DiabetesPedigreeFunction', 'value'),
         Input('Age', 'value'),
         Input('BMI_levels', 'value'),
         Input('Glucose_levels', 'value')
        ]
)
def predict(*inputlist):
    if(inputlist[0]=='SVM'):
        return 'Result: {}'.format(svm_cv.best_estimator_.predict(np.array(inputlist[1:]).reshape(1,-1)))
    else:
        return 'Result: {}'.format(LR_cv.best_estimator_.predict(np.array(inputlist[1:]).reshape(1,-1)))

In [45]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Oct/2021 02:49:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:49:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:49:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:49:42] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:49:42] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:49:42] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:49:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:49:43] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

127.0.0.1 - - [27/Oct/2021 02:49:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

...

127.0.0.1 - - [27/Oct/2021 02:49:59] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

127.0.0.1 - - [27/Oct/2021 02:50:01] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Toqa\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Toqa\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

127.0.0.1 - - [27/Oct/2021 02:50:02] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/Oct/2021 02:50:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 02:50:09] "POST /_dash-update-component HTTP/1.1" 200 -
